In [1]:
import os
import pandas as pd 
import glob
import numpy as np
import torch
from torch.utils.data import DataLoader
import transformers
import torch.nn as nn
from tqdm import tqdm
from prepare_invoice_ner_dataset import label_idx_dict
from prepare_invoice_ner_dataset import split_tokenize_label_dataset, split_tokenize_label_file, form_input
from print_ner_tag import print_ner_labels, print_ner_labels_detokenized


In [2]:
test_split_path = './split/test.txt'

from transformers import AutoTokenizer, BertTokenizer
bert_path = '../kaggle_ner/huggingface-bert/bert-base-uncased/'
#model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(bert_path, do_lower_case=True)

In [3]:
config = {'MAX_LEN':128,
          'tokenizer': tokenizer,
          'batch_size':32,
          'Epoch': 1,
          'device': 'cuda' if torch.cuda.is_available() else 'cpu',
          'model_name':'model1_bert_base_uncased_3_epochs.bin'
         }

In [4]:
model = transformers.BertForTokenClassification.from_pretrained('bert-base-uncased',  num_labels = len(label_idx_dict))
model = nn.DataParallel(model)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [5]:
#model.load_state_dict(torch.load(config['model_name']))
#model = model.to(config['device'])


In [6]:
mode = model.eval()

In [7]:
final_test_split_and_tokenized_file_list, final_test_split_and_tokenized_labels, final_test_split_word_id_list, final_test_split_token_ids_list = split_tokenize_label_dataset(test_split_path, tokenizer, overlap=0)

test_prod_input = form_input(final_test_split_and_tokenized_file_list, final_test_split_and_tokenized_labels, 
                              final_test_split_token_ids_list, final_test_split_word_id_list, config, data_type='test')


In [8]:
invoice_number = 1

if_estimation = True
estimation_list = []
gt_list = []
print("Sample Invoice {}".format(invoice_number))
print()
for sample_idx in range(len(test_prod_input)):
    #print("Estimation")
    
    
    #print("sample_idx: {}".format(sample_idx))
    dataset = test_prod_input[sample_idx]

    batch_input_ids = torch.unsqueeze(dataset['ids'], 0).to(config['device'], dtype = torch.long)
    batch_att_mask = torch.unsqueeze(dataset['att_mask'], 0).to(config['device'], dtype = torch.long)
    batch_tok_type_id = torch.unsqueeze(dataset['tok_type_id'], 0).to(config['device'], dtype = torch.long)
    batch_target = torch.unsqueeze(dataset['target'], 0).to(config['device'], dtype = torch.long)
    
    bert_tokens = dataset['bert_tokens']
    bert_token_word_ids = dataset['bert_token_word_ids']
    
    output = model(batch_input_ids, 
                       token_type_ids=None,
                       attention_mask=batch_att_mask,
                       #labels=batch_target
              )
    sample_ner_result = output['logits'].detach().numpy()[0]
    sample_estimated_tags = np.argmax(sample_ner_result, -1)
    sample_tag_labels = test_prod_input[sample_idx]['target'].numpy()   
    
    sample_estimated_tag_labels = np.array([list(label_idx_dict.keys())[label_idx] for label_idx in list(sample_estimated_tags)])
    sample_ground_truth_tag_labels = np.array([list(label_idx_dict.keys())[label_idx] for label_idx in list(sample_tag_labels)])
        
    estimation_list.append((bert_tokens, sample_estimated_tag_labels[:len(bert_tokens)], bert_token_word_ids))
    gt_list.append((bert_tokens, sample_ground_truth_tag_labels[:len(bert_tokens)], bert_token_word_ids))
    
    if(len(bert_tokens)!=128):     
        print("Estimation")
        for estimation_tuple in estimation_list:
            print_ner_labels_detokenized(estimation_tuple[0], estimation_tuple[1], estimation_tuple[2])
        print()
        print()
        print("Ground truth")
        for gt_tuple in gt_list:
            print_ner_labels_detokenized(gt_tuple[0], gt_tuple[1], gt_tuple[2])
        print()
        print()
        print()
        
        gt_list = []
        estimation_list = []
        invoice_number += 1
        

        if(invoice_number==6):
            break
            
        print("Sample Invoice {}".format(invoice_number))

Sample Invoice 1

Estimation
tax invoice invoice no . sm/18 - 19/01159 delivery note sm/18 - 19/01159 supplier 's ref . ( k ) dated 17-may-18 mode / terms of payment other reference(s ) shall marketing shop no.6 llyods chamber barne road 409 mangalwar peth pune gstin / uin : 27adxps3921j1zt state name : maharashtra code : 27 contact : 20 - - 26052747 e - mail : shahmarketing4@gmail.com blyer siddharth properties - s .b. road geetai sankul 2nd floor paud road near reliance fresh pune . gstin / uin : 27aazfs3607c1zy state name : maharashtra code : 27 buyer 's order no . dated despatch document no . delivery note date 17-may-18 destination despatched through terms of delivery ' skybay " mhalunge pune .. mr.sameer;-8308846229 mr.anil:-9689921859 po no;-11887 contact person contact e - mail : mr. thube / mane : 8308824752 : nilesh.mane@saarrthigroup.co.in description of goods hsn / sac quantity rate gst kate per disc . % ancount si no . 1 g.m. ballvalve 01-feb " 2 upvc elbow 1 01-feb " 90 d

Estimation
gst invoice ( original for recipient ) m invoice no . e - way bill no . sm/20 - 21/01746 2.5122e+11 delivery note dated 07-sep-20 mode / terms of payment shah marketing ( mangalwar peth ) ( bath division ) llyods chambers shop no.5 & 6 409 mangalwar peth barne road pune gstin / uin : 27adxps3921j1zt state name : maharashtra code : 27 contact : 20 - 26052647 / 26052747 e - mail : sales.shah marketing@gmail.com supplier 's ref . other reference(s ) samy homes buyer 's order no . dated despatch document no . delivery note date buyer siddharth value homes llp geetai sankul 2nd floor paud road near reliance fresh pune . gstin / uin : 27aczfs0314c1z4 state name : maharashtra code : 27 despatched through destination terms of delivery sawy homes opp to orriental hospital hinjewadi . 9689908626.(po no;-472 ) contact person : nilesh mane contact : 7350000353 e - mail : nilesh.m@saarrthigroup.com description of goods hsn / sac rate si no . amount gst quantity rate per disc . % 1 veiga 

Estimation
gst invoice ( original for recipient ) invoice no . sm/19 - 20/00500 delivery note e - way bill no . dated 22-apr-19 mode / terms of payment other reference(s ) shah marketing ( mangalwar peth ) shop no.6 llyods chamber barne road 409 mangalwar peth pune gstin / uin : 27adxps3921j1zt state name : maharashtra code : 27 contact : 20 - - 26052747 e - mail : shahmarketing4@gmail.com buyer siddharth properties geetai sank ul 2nd floor paud road near reliance fresh pune . gstin / uin : 27aazfs3607c1zy state name : maharashtra code : 27 supplier 's ref . quotation245 buyer 's order no . quotation245 despatch document no . dated 22-apr-19 delivery note date despatched through destination terms of delivery saarrthi swadesh transportation will be aranged by siddharth pro contact person contact e - mail : mr. thube / mane : 8308824752 : nilesh.mane@saarrthigroup.co .in description of goods hsn / sac quantity rate si no . amount gst rate per disc . % 1 tajay pvc nanhni trap wio jali 110

Estimation
aviva doors factory address:- gat no.255b plot no.26/27 jyotiba nagar talawade pune-411062 registered office : - e-83 adinath society pune - satara road pune - 411037 contact : - 20 - 24269505 mob:- 91 8446380632 - email : - avivadoor@gmail.com tax invoice original invoice no . : ti-356/19 - 20 buyer party m / s . : siddharth properties . eden court model colony pune date : 13-08-2019 po / ref no . : 3516-bath ch . no . : 2045 pune place of supply : 27-maharashtra gstin no . : 27aazfs3607c1zy ch date . : 12-08-2019 vehicle no . : mh14hg2249 site name : del . add : success squaer po - no - saa / mi/2019/13208 karve nagar despatch thr . : datta srno product name hsn / sac nos total rate gst% amount 4418 26 26 nos 4100 18 106600 1 32mm bsl door toilet - gas - toilet - tr 5 . square porno received sub total 106600 gstin no . : 27aarfa8194j1zp bank name : indian bank bank a / c . no . : 894914709 rtgs / ifsc code : idibooono12 total gst : ninteen thousand one hundred eighty eight

Estimation
aviva doors factory address:- gat no.255b plot no.26/27 jyotiba nagar talawade pune-411062 istered office : - e-83 adinath society pune - satara road pune - 411037 contact : - 20 - 24269505 mob:- 91 8446380632 - email : - avivadoor@gmail.com tax invoice original buye invoice no . : ti-500/18 - 19 m / s . date : 25-12-2018 dharth real ventures llp . ce no-60 1 & 602 mccla trad tower icc complex \t road - pune mo-7745895778 po / ref no . : 3084 488 a© ch . no . : 1029 plac . : 27-maharashtra : 27acmfs9508b1zx ch date . : gst 25-12-2018 vehicle no . : mh14em2398 erenity site del. despatch thr . : datta ud road othrud o - no-9975258821 po : nofann ) 2018-hig srnc product name hsn / sac nos total rate gst% amount 3sl door - md 4418 9 9 sq . ft . 14322 18 128898 site name : received seventy me date:- 8 1111a mu gate entry code.1.9.3.6 . ........ velcie no ... mh:12.em.2398 out time ............ gst 27aarfa8194j1zp sub total 128898 bank ank rtgs : indian bank : 894914709 : idibooon